# **Practica 2.2**
#### _Alberto García Doménech - Pablo Daurell Marina_ (Grupo 10)
***

## Parte 2 (Recuperación de información)

In [32]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np

Para esta práctica vamos a utilizar el dataset ```20 Newsgroup``` de Scikit-learn, que contiene textos de un foro sobres distintos temas muy variados.

In [33]:
# Cargamos el dataset
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

Training texts: 11314
Test texts: 7532


Podemos ver algunos ejemplos de los textos que hay en el dataset:

In [34]:
print('MENSAJE:\n',train_data.data[120])
print('CLASE: ', train_data.target_names[train_data.target[120]])

MENSAJE:
 From: shd2001@andy.bgsu.edu (Sherlette Dixon)
Subject: Christianity & Atheism:  an update
Organization: BGSU
Lines: 32

First, I would like to thank all who sent me their opinions on the matter
at hand.  All advice was taken to heart, if not directly used.  My friend
found out about the matter quite accidently.  After reading some of my
mail, I quit from the mail reader & went about my business.  I must have
trashed my mail improperly, because he got on the same terminal the next
day & saw my old messages.  He thought they were responses to a post he
placed in alt.atheism earlier that week, so he read some of them before
realizing that they were for me.  I got a message from him the next day; he
apologized for reading my mail & said that he did not want to appear to be
a snoop.  He said that he would be willing to talk to me about his views &
didn't mind doing so, especially with a friend.  So we did.  I neither
changed his mind nor did he change mine, as that was not the poi

In [35]:
print('MENSAJE:\n',train_data.data[42])
print('CLASE: ', train_data.target_names[train_data.target[42]])

MENSAJE:
 From: ab245@cleveland.Freenet.Edu (Sam Latonia)
Subject: Re: Need phone number for Western Digital (ESDI problem)
Organization: Case Western Reserve University, Cleveland, Ohio (USA)
Lines: 5
NNTP-Posting-Host: slc10.ins.cwru.edu


Western Digital 1-800-832-4778.....Sam
-- 
Gosh..I think I just installed a virus..It was called MS DOS6...
Don't copy that floppy..BURN IT...I just love Windows...CRASH...

CLASE:  comp.sys.ibm.pc.hardware


### A) Bolsa de palabras binaria

Convertimos el conjunto de datos en una bolsa de palabras, binaria y con n-gramas (1,1), usaremos como vocabulario el diccionario ```words.txt``` (https://github.com/dwyl/english-words/blob/master/words.txt)

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

In [37]:
# Pasamos el diccionario a una lista
with open('words.txt') as f:
    dictionary = f.read().splitlines()

# Creamos la bolsa de palabras
vectorizer = CountVectorizer(binary=False, ngram_range=(1,1), vocabulary=dictionary, stop_words='english')  

# Entrenamos al modelo
train_vector_data = vectorizer.fit_transform(train_data.data)

feature_names = vectorizer.get_feature_names()
print(len(feature_names))

466551


In [38]:
np.shape(train_vector_data)

(11314, 466551)

In [39]:
def select_messages(data, target, target_names, n):
    '''
    Selecciona n mensajes de cada clase
    Return: Matriz con una fila por cada clase y tres mensajes por cada fila
    '''
    messages = []
    # Recorremos todas las clases
    for i in range(len(target_names)):
        # Seleccionamos las posiciones correspondientes a los mensajes de la clase i
        indices = np.asarray(np.where(target == i)).ravel()

        # Cogemos los n primeros mensajes de la clase i
        aux = np.take(data, indices)[0:n]
        messages.append(aux)
    
    return np.array(messages)

def vectorize_messages(data, vectorizer, tfider=None):
    '''
    Recibe una matriz de textos y convierte cada texto a una bolsa de palabras,
    usando un vectorizador previamente entrenado (y pasandolo a tfidf si se pide)
    Return: Matriz con una fila por cada clase y en cada fila otra matriz con los mensajes vectorizados
    '''
    # Recorremos todas las filas (una por cada clase)
    vector_data = []
    for i in range(np.shape(data)[0]):
        aux = vectorizer.transform(data[i, :]) # Vectorizamos el mensaje
        
        if tfider:
            aux = tfider.transform(aux) # Lo convertimos a tfidf
            
        vector_data.append(aux)
        
    return np.array(vector_data)

In [40]:
# Seleccionamos, del conjunto de test, 3 mensajes de cada una de las 20 clases:
messages = select_messages(test_data.data, test_data.target, test_data.target_names, 3)

In [41]:
# Vectorizamos los mensajes seleccionados
vector_messages = vectorize_messages(messages, vectorizer)

#### Similitud del coseno:

Vamos a usar como consultas los mensajes seleccionados para recuperar los mensajes del conjunto de entrenamiento que más se parezcan. Con esto veremos que clases se recuperan para cada consulta y podremos calcular la precisión obtenida para cada clase.    

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_precission(vector_data, target, messages, target_names, X):
    '''
    Calcula la similitud del coseno de cada mensaje con todos los mensajes de "data" y
    y calcula la precision de cada clase, a un nivel de exhaustividad X.
    '''
    for i in range(len(messages)): # Recorremos cada clase
        precission = 0
        for m in messages[i]: # Recorremos cada mensaje
            # Calculamos la similitud del coseno de m con los textos de entrenamiento
            cos_sim = cosine_similarity(m, vector_data).ravel()

            # Ordenamos los resultados y cogemos los indices de los X mayores resultados
            max_args = np.flip(np.argsort(cos_sim))[0:X]

            #  max_values = np.take(cos_sim, max_args)         
            #  Recuperamos los mensajes con mayor similitud
            # retrieved_docs = np.take(data, max_args)

            # Identificamos la clase de los mensajes con mayor similitud
            retrieved_classes = np.take(target, max_args)

            # Calculamos la precision (%) para este mensaje y lo acumulamos a las anteriores
            precission += (np.count_nonzero(retrieved_classes == i) / len(retrieved_classes)) * 100


        # Calculamos la media de los porcentajes obtenidos
        precission /= np.shape(vector_messages[i])[0]
        
        print('Class {}:'.format(i), target_names[i], ' Precission: ', precission)

Usando la función que acabamos de definir, vamos a calcular la precisión media de cada clase a un nivel de exhaustividad 3 (es decir, recuperando 3 mensajes del conjunto de entrenamiento) y a nivel 10 (recuperando 10 mensajes):

In [43]:
# Precision con nivel de exhaustividad = 3
cosine_similarity_precission(train_vector_data, train_data.target, vector_messages, train_data.target_names, 3)

Class 0: alt.atheism  Precission:  33.333333333333336
Class 1: comp.graphics  Precission:  22.222222222222218
Class 2: comp.os.ms-windows.misc  Precission:  22.222222222222218
Class 3: comp.sys.ibm.pc.hardware  Precission:  55.55555555555555
Class 4: comp.sys.mac.hardware  Precission:  0.0
Class 5: comp.windows.x  Precission:  33.33333333333333
Class 6: misc.forsale  Precission:  77.77777777777777
Class 7: rec.autos  Precission:  33.333333333333336
Class 8: rec.motorcycles  Precission:  88.88888888888887
Class 9: rec.sport.baseball  Precission:  77.77777777777777
Class 10: rec.sport.hockey  Precission:  66.66666666666667
Class 11: sci.crypt  Precission:  55.55555555555554
Class 12: sci.electronics  Precission:  44.444444444444436
Class 13: sci.med  Precission:  77.77777777777777
Class 14: sci.space  Precission:  77.77777777777777
Class 15: soc.religion.christian  Precission:  77.77777777777777
Class 16: talk.politics.guns  Precission:  55.55555555555554
Class 17: talk.politics.mideast 

In [44]:
# Precision con nivel de exhaustividad = 10
cosine_similarity_precission(train_vector_data, train_data.target, vector_messages, train_data.target_names, 10)

Class 0: alt.atheism  Precission:  23.333333333333332
Class 1: comp.graphics  Precission:  20.0
Class 2: comp.os.ms-windows.misc  Precission:  30.0
Class 3: comp.sys.ibm.pc.hardware  Precission:  50.0
Class 4: comp.sys.mac.hardware  Precission:  6.666666666666667
Class 5: comp.windows.x  Precission:  30.0
Class 6: misc.forsale  Precission:  56.666666666666664
Class 7: rec.autos  Precission:  30.0
Class 8: rec.motorcycles  Precission:  73.33333333333333
Class 9: rec.sport.baseball  Precission:  76.66666666666667
Class 10: rec.sport.hockey  Precission:  56.666666666666664
Class 11: sci.crypt  Precission:  36.666666666666664
Class 12: sci.electronics  Precission:  50.0
Class 13: sci.med  Precission:  53.333333333333336
Class 14: sci.space  Precission:  73.33333333333333
Class 15: soc.religion.christian  Precission:  50.0
Class 16: talk.politics.guns  Precission:  40.0
Class 17: talk.politics.mideast  Precission:  26.666666666666668
Class 18: talk.politics.misc  Precission:  23.33333333333

>Hay una gran diferencia entre los valores de precision medios de las diferentes clases, siendo la mayor diferencia de un 88.88% con un nivel de exhaustividad 3 y de un 70% con un nivel de exhaustividad de 10.    
Esta clara diferencia puede venir por la ausencia de terminos especificos de la clase que estamos buscando en las consultas lo que puede llevar a una confusión entre clases. Por ejemplo, la clase con menor precisión es _comp.sys.mac.hardware_ con un 0% de precisión con nivel 3 de exhaustividad y un 6.67% con nivel 10, este nivel tan bajo de precisión seguramente se deba a que las consultas elegidas se confunden con otras clases que traten de temas relacionados con informatica, como es el caso de _comp.sys.ibm.pc.hardware_, _comp.os.ms-windows.misc_, etc.            
Mientras que las clases que mejor precisión media tienen (_rec.motorcycles , rec.sport.basketball_) seguramente se deba a que las consultas tienen términos más específicos que las clasifiquen como las clases a las que pertenecen.

- Seleccionamos un mensaje de la clase con peor precision (_com.sys.mac.hardware_) y vemos que mensajes se han recuperado con él para poder observar más en detalle que puede ser el causante de los malos resultados:

In [45]:
def worst_class_precission(data, vector_data, target, message, vector_message, target_names, X):
    '''
    Dado un mensaje, recupera los X mensajes de "data" 
    y los muestra junto a sus clases correspondientes
    '''
    # Calculamos la similitud del coseno de m con los textos de entrenamiento
    cos_sim = cosine_similarity(vector_message, vector_data).ravel()

    # Ordenamos los resultados y cogemos los indices de los X mayores resultados
    max_args = np.flip(np.argsort(cos_sim))[0:X]

    # Identificamos la clase de los mensajes con mayor similitud
    retrieved_classes = np.take(target, max_args)
    retrieved_classes = np.take(target_names, retrieved_classes)
    #  Recuperamos los mensajes con mayor similitud
    retrieved_docs = np.take(data, max_args)
    
    print('CONSULTA:\n',message)
    print('*************************************')
    print('Clases recuperadas:', retrieved_classes)
    for i, d in enumerate(retrieved_docs):
        print('*************************************')
        print('Mensaje recuperado {}:\n'.format(i), d)
            
        

In [46]:
# Mostramos los mesajes recuperados para la segunda consulta de la clase com.sys.mac.hardware
worst_class_precission(train_data.data, train_vector_data, train_data.target, messages[4][1], vector_messages[4][1], train_data.target_names, 3)

CONSULTA:
 From: s9131783@valiant.vut.EDU.AU (Robert B Harvey)
Subject: Disabling the Eject on a Mac SE
Organization: Victoria University Of Technology, Melbourne, Australia
Lines: 13

I'm trying to find a program that will stop the Macs from spitting out
their Boot Disk. I was told one exists but I can't find it.

Anyone know where I can find it?

Thanks

Robert Harvey
Duty Programmer
Information Technology
Victoria University

s9131783@valiant.vut.edu.au

*************************************
Clases recuperadas: ['comp.windows.x' 'misc.forsale' 'sci.electronics']
*************************************
Mensaje recuperado 0:
 From: queloz@bernina.ethz.ch (Ronald Queloz)
Subject: Hypercard for UNIX
Organization: Swiss Federal Institute of Technology (ETH), Zurich, CH
Lines: 10

Hi netlanders,

Does anybody know if there is something like Macintosh Hypercard for any UNIX 
platform?


Thanks in advance


Ron.

*************************************
Mensaje recuperado 1:
 From: gt4661a@prism

Las tres clases con las que más se ha confundido esta consultas son con _comp.windows.x, misc.forsale, sci.electronics_, las tres tienen una cosa en común: la tecnología: ya sea porque dos de esas clases están basadas en tecnolgía y ordenadores o porque el mensaje de clase _misc.forsale_ hable sobre la venta de un ordenador. Los tres mensajes vienen de partes de instituciones de tecnología(ya sean universidades o institutos) y es lógico la confusión de clase entre la consulta y los resultados. Además el mensaje de consulta hace muy poca referencia a macs mientras que usa bastantes palabras de termino tçecnologico, y particularmente de ordenadores.

### B) Bolsa de palabras con TF-IDF

Vamos a repetir el mismo proceso del apartado anterior, pero esta vez vamos a vectorizar los mensajes con TF-IDF.

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

# Pasamos el diccionario a una lista
with open('words.txt') as f:
    dictionary = f.read().splitlines()

# Creamos la bolsa de palabras
vectorizer = CountVectorizer(binary=False, ngram_range=(1,1), vocabulary=dictionary, stop_words='english')  

# Entrenamos al modelo
train_vector_data = vectorizer.fit_transform(train_data.data)
feature_names = vectorizer.get_feature_names()

# Convertimos la frecuencia a TF-IDF
tfider = TfidfTransformer()
train_preprocessed = tfider.fit_transform(train_vector_data)

In [17]:
np.shape(train_preprocessed)

(11314, 466551)

In [18]:
# Usamos los mismos mensajes seleccionados anteriormente
# Los vectorizamos, esta vez usando TF-IDF
vector_messages = vectorize_messages(messages, vectorizer, tfider)

In [19]:
# Precision con nivel de exhaustividad = 3
cosine_similarity_precission(train_preprocessed, train_data.target, vector_messages, train_data.target_names, 3)

Class 0: alt.atheism  Precission:  22.222222222222218
Class 1: comp.graphics  Precission:  11.111111111111109
Class 2: comp.os.ms-windows.misc  Precission:  11.111111111111109
Class 3: comp.sys.ibm.pc.hardware  Precission:  66.66666666666667
Class 4: comp.sys.mac.hardware  Precission:  33.33333333333333
Class 5: comp.windows.x  Precission:  33.33333333333333
Class 6: misc.forsale  Precission:  44.444444444444436
Class 7: rec.autos  Precission:  44.444444444444436
Class 8: rec.motorcycles  Precission:  77.77777777777777
Class 9: rec.sport.baseball  Precission:  100.0
Class 10: rec.sport.hockey  Precission:  100.0
Class 11: sci.crypt  Precission:  55.55555555555554
Class 12: sci.electronics  Precission:  44.444444444444436
Class 13: sci.med  Precission:  77.77777777777777
Class 14: sci.space  Precission:  100.0
Class 15: soc.religion.christian  Precission:  77.77777777777777
Class 16: talk.politics.guns  Precission:  66.66666666666666
Class 17: talk.politics.mideast  Precission:  77.7777

In [20]:
# Precision con nivel de exhaustividad = 10
cosine_similarity_precission(train_vector_data, train_data.target, vector_messages, train_data.target_names, 10)

Class 0: alt.atheism  Precission:  26.666666666666668
Class 1: comp.graphics  Precission:  30.0
Class 2: comp.os.ms-windows.misc  Precission:  30.0
Class 3: comp.sys.ibm.pc.hardware  Precission:  60.0
Class 4: comp.sys.mac.hardware  Precission:  13.333333333333334
Class 5: comp.windows.x  Precission:  40.0
Class 6: misc.forsale  Precission:  30.0
Class 7: rec.autos  Precission:  46.666666666666664
Class 8: rec.motorcycles  Precission:  76.66666666666667
Class 9: rec.sport.baseball  Precission:  93.33333333333333
Class 10: rec.sport.hockey  Precission:  100.0
Class 11: sci.crypt  Precission:  36.666666666666664
Class 12: sci.electronics  Precission:  60.0
Class 13: sci.med  Precission:  63.333333333333336
Class 14: sci.space  Precission:  80.0
Class 15: soc.religion.christian  Precission:  60.0
Class 16: talk.politics.guns  Precission:  33.333333333333336
Class 17: talk.politics.mideast  Precission:  50.0
Class 18: talk.politics.misc  Precission:  46.666666666666664
Class 19: talk.relig